Neurons are the simplest units of our nervous system. Complex communication between neurons is what gives rise to all brain functions. Scientists in the 19th century believed that neurons communicate using electricity, but it was not until 1974 that Louis Edouard Lapique came up with a simple mathematical model of how neurons send electric signals to each other. This model is called the Leaky Integrate-and-Fire Neuron (LIF). The components of the model are:

1. Membrane Potential (V) which, when multiplied by the Capacitance (C), gives the current electric charge of the neuron. Capacitance is how much charge a neuron can store (in the form of charged ions), and potential is the energy that can push the charges in the neuron for ions (electric signal) to flow.

2. Resting Potential ($E_L$) which is the membrane potential at rest. When this potential goes above a certain threshold ($V_t$) by the action of external input current (I), the neuron fires, and a
spike is generated.

3. External Input (I) which describes the current that feeds into the neuron, part of which is used to increase the neurons’ electrical potential and thus ion charge, but some of it gets leaked outside the neuron. The leakage is determined by Leak Conductance ($g_L$) which is the tendency of the neuron to leak charge when it is too charged, i.e. V >> $E_L$.

4. Diving the capacitance by the leak conductance values gives rise to a property called the Time Constant ($𝛕_m$), which is the time it takes for a neuron to charge and fire a spike, controlled by its ability to store charge versus its tendency to leak it off which delays the charging process and thus the increase of V until it passes V_t to fire a spike.

Now that we presented the model components, let’s discuss the equations that combine them together to explain the model.

$C_m dV/dt = -g_L(V-E_L)  + I$

$𝜏_m dV/dt = -(V-E_L)  + I / g_L$

The first equation expresses the change in neuron charge over time. This is directly proportional to the external current and decreases as the leak conductance increases because more charge is lost. The leak increases the more the neuron is charged far from its resting state. The second equation is a different version of the first by dividing it over $g_L$. It simply shows that the neuron gets charged over time by the external current with a leak, and the change in charge is inversely proportional to the time constant of the neuron. Therefore, the bigger 𝜏m gets, the longer it takes to generate a spike. The interactive figure below shows this effect.

**Imports**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import ipywidgets as widgets

**Figure helpers**

In [2]:
# Disable logging for matplotlib font manager
logging.getLogger('matplotlib.font_manager').disabled = True

# Set the figure format for inline backend
%config InlineBackend.figure_format = 'retina'

# Use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

# Define a layout for widgets
custom_layout = widgets.Layout()

def standard_parameters(**optional_params):
    params = {}

    # Typical neuron parameters
    params['threshold_voltage'] = -55.     # spike threshold [mV]
    params['reset_voltage'] = -75.  # reset potential [mV]
    params['membrane_tau'] = 10.     # membrane time constant [ms]
    params['leak_conductance'] = 10.       # leak conductance [nS]
    params['initial_voltage'] = -75.   # initial potential [mV]
    params['leak_potential'] = -75.      # leak reversal potential [mV]
    params['refractory_duration'] = 2.       # refractory time (ms)

    # Simulation parameters
    params['total_time'] = 400.  # Total duration of simulation [ms]
    params['time_step'] = .1   # Simulation time step [ms]

    # External parameters if any
    for key in optional_params:
        params[key] = optional_params[key]

    params['time_vector'] = np.arange(0, params['total_time'], params['time_step'])  # Vector of discretized time points [ms]

    return params

neuron_params = standard_parameters()

def display_voltage_trajectory(params, voltage, spike_train):
    """
    Plot trajectory of membrane potential for a single neuron

    Expects:
    params       : parameter dictionary
    voltage      : voltage trajectory
    spike_train  : spike train

    Returns:
    figure of the membrane potential trajectory for a single neuron
    """

    threshold = params['threshold_voltage']
    time_step, time_vector = params['time_step'], params['time_vector']
    if spike_train.size:
        spike_indices = (spike_train / time_step).astype(int) - 1
        voltage[spike_indices] += 20  # draw nicer spikes

    plt.plot(params['time_vector'], voltage, 'b', color = 'g')
    plt.axhline(threshold, 0, 1, color='k', ls='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (mV)')
    plt.legend(['Membrane\npotential', r'Threshold V$_{\mathrm{th}}$'],
               loc=[1.05, 0.75])
    plt.ylim([-80, -40])
    plt.show()


**Model Helpers**

In [3]:
def simulate_neuron(params, input_current, pulse=False):
    """
    Simulate the LIF dynamics with external input current

    Args:
      params         : parameter dictionary
      input_current  : input current [pA]. The injected current here can be a value
                       or an array
      pulse          : boolean. If True, use a current pulse

    Returns:
      voltage_trace  : membrane potential
      spike_times    : spike times
    """

    # Set parameters
    threshold, reset_voltage = params['threshold_voltage'], params['reset_voltage']
    membrane_tau, leak_cond = params['membrane_tau'], params['leak_conductance']
    initial_voltage, leak_rev_potential = params['initial_voltage'], params['leak_potential']
    time_step, time_vector = params['time_step'], params['time_vector']
    total_time_points = time_vector.size
    refractory_duration = params['refractory_duration']

    # Initialize voltage
    voltage_trace = np.zeros(total_time_points)
    voltage_trace[0] = initial_voltage

    # Set current time course
    input_current = input_current * np.ones(total_time_points)

    # If current pulse, set beginning and end to 0
    if pulse:
        input_current[:int(len(input_current) / 2) - 1000] = 0
        input_current[int(len(input_current) / 2) + 1000:] = 0

    # Loop over time
    recorded_spikes = []  # record spike times
    refractory_counter = 0.  # the count for refractory duration

    for time_idx in range(total_time_points - 1):

        if refractory_counter > 0:  # check if in refractory period
            voltage_trace[time_idx] = reset_voltage  # set voltage to reset
            refractory_counter = refractory_counter - 1 # reduce running counter of refractory period

        elif voltage_trace[time_idx] >= threshold:  # if voltage over threshold
            recorded_spikes.append(time_idx)  # record spike event
            voltage_trace[time_idx] = reset_voltage  # reset voltage
            refractory_counter = refractory_duration / time_step  # set refractory time

        # Calculate the increment of the membrane potential
        delta_v = (-(voltage_trace[time_idx] - leak_rev_potential) + input_current[time_idx] / leak_cond) * (time_step / membrane_tau)

        # Update the membrane potential
        voltage_trace[time_idx + 1] = voltage_trace[time_idx] + delta_v

    # Get spike times in ms
    spike_times = np.array(recorded_spikes) * time_step

    return voltage_trace, spike_times


**Interactive Widget**

In [4]:
# Reminder: Ensure this cell is executed to activate the interactive widget!

# Set the layout width
custom_layout.width = '450px'

# Create interactive widgets
@widgets.interact(
    direct_current=widgets.FloatSlider(value=50., min=0., max=300., step=10., layout=custom_layout),
    membrane_time_const=widgets.FloatSlider(value=10., min=2., max=20., step=2., layout=custom_layout)
)

def display_neuron_behavior(direct_current=200., membrane_time_const=10.):
    """
    Visualize the response of an LIF neuron for given parameters.

    Args:
    - direct_current     : Direct current input.
    - membrane_time_const: Membrane time constant.
    """
    # Get default parameters and update membrane_time_const
    neuron_params_updated = standard_parameters(total_time=100.)
    neuron_params_updated['membrane_tau'] = membrane_time_const

    # Simulate the LIF neuron
    voltage_output, spike_output = simulate_neuron(neuron_params_updated, input_current=direct_current)

    # Plot the voltage trace
    display_voltage_trajectory(neuron_params_updated, voltage_output, spike_output)
    plt.show()


interactive(children=(FloatSlider(value=50.0, description='direct_current', layout=Layout(width='450px'), max=…

We can see that increasing tau_m, the membrane time constant, makes us need more I_dc for enough effective charge to accumulate to cause an action potential spike.

Now let's add one more layer of real-world complexity into the model. In-vivo brain electricity is not as uniform as DC current is. It is noisy and includes fluctuations. The consider this we add Gaussian noise to electric current in our model. So, the interactive figure below explores the effect of adding Gaussian noise to the input current of neurons and the extent to which the variance of that noisy current influences spiking.

In [5]:
def generate_GWN(params, noise_mean, noise_std, random_seed=False):
    """
    Function to produce Gaussian white noise input.

    Args:
    params        : Dictionary of parameters.
    noise_mean    : Baseline of the noise (mean).
    noise_std     : Amplitude of the noise (standard deviation).
    random_seed   : Seed for random number generation. Can be int or boolean.
                    Using the same seed will yield the same sequence of random numbers.

    Returns:
    noise_input   : Gaussian white noise input.
    """

    # Extract simulation parameters
    time_step, time_points = params['time_step'], params['time_vector']
    total_points = time_points.size

    # Set the random seed
    if random_seed:
        np.random.seed(seed=random_seed)
    else:
        np.random.seed()

    # Generate Gaussian white noise
    # The division by 1000 is for unit conversion to seconds.
    noise_input = noise_mean + noise_std * np.random.randn(total_points) / np.sqrt(time_step / 1000.)

    return noise_input


In [6]:
custom_layout.width = '460px'

@widgets.interact(
    mean_noise=widgets.FloatSlider(200., min=100., max=310., step=5., layout=custom_layout),
    std_noise=widgets.FloatSlider(2.5, min=0., max=5.5, step=.5, layout=custom_layout)
)

def compare_GWN_with_LIF(mean_noise, std_noise):
    neuron_params = standard_parameters(duration=102.)
    GWN_input = generate_GWN(neuron_params, noise_mean=mean_noise, noise_std=std_noise)
    potential_output, spike_output = simulate_neuron(neuron_params, input_current=GWN_input)

    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(neuron_params['time_vector'][::3], GWN_input[::3], 'b')
    plt.xlabel('Time (ms)')
    plt.ylabel(r'Noisy Input current (pA)')
    plt.subplot(122)
    display_voltage_trajectory(neuron_params, potential_output, spike_output)
    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=200.0, description='mean_noise', layout=Layout(width='460px'), max=310…

Here we can see that increasing the noise level in the input current (standard deviation of the Gaussian noise) helps neurons with lower average DC component of the input current to reach their threshold potential and fire compared to small fluctuations that don't help push the voltage to their firing threshold. It is important to note that this is an important property in real-world neurons which should explain any discrepancy between simulated spiking rates from just DC current and the actual neural behavior. We can see this more clearly in the interactive plot below.

In [7]:
custom_layout.width = '460px'

@widgets.interact(
    noise_std=widgets.FloatSlider(3.0, min=0., max=6.5, step=0.5, layout=custom_layout)
)

def analyze_std_effect_on_fI(noise_std):
    neuron_params = standard_parameters(duration=1020.)
    current_avg = np.arange(105., 410., 10.)
    spike_counts = np.zeros(len(current_avg))
    spike_counts_dc = np.zeros(len(current_avg))

    for i in range(len(current_avg)):
        GWN_current = generate_GWN(neuron_params, noise_mean=current_avg[i], noise_std=noise_std, random_seed=2025)
        potential_output, recorded_spikes = simulate_neuron(neuron_params, input_current=GWN_current)
        potential_dc, spikes_dc = simulate_neuron(neuron_params, input_current=current_avg[i])
        spike_counts[i] = len(recorded_spikes)
        spike_counts_dc[i] = len(spikes_dc)

    # Visualize the F-I curve: Output firing rate against input mean.
    plt.figure()
    plt.plot(current_avg, spike_counts, 'k', label=r'Noise Std=%.2f' % noise_std, color = 'r')
    plt.plot(current_avg, spike_counts_dc, 'k--', alpha=0.5, lw=4, dashes=(2, 2), label='DC Input')
    plt.ylabel('Spike Count')
    plt.xlabel('Input Current (pA)')
    plt.legend(loc='best')
    plt.show()


interactive(children=(FloatSlider(value=3.0, description='noise_std', layout=Layout(width='460px'), max=6.5, s…

We can see that when we set sigma to zero, the spike count follows a pattern similar to that with a DC input current, where the threshold of the injected current that causes a spike is high. However, as we increase signal noise, smaller currents become able to induce spikes.